# БД SQLite. Переработанный материал из книги 'Секреты Python Pro' (Practices of the Python Pro), автор Дейн Хиллард

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

In [2]:
class DatabaseManager:
    def __init__(self, database_filename):
        self.connection = sqlite3.connect(database_filename)

    def __del__(self):
        self.connection.close()

    def _execute(self, statement, values=None):
        with self.connection:
            cursor = self.connection.cursor()
            cursor.execute(statement, values or [])
            return cursor

    def create_table(self, table_name, columns):
        columns_with_types = [
            f'{column_name} {data_type}'
            for column_name, data_type in columns.items()
        ]
        self._execute(
            f'''
            CREATE TABLE IF NOT EXISTS {table_name}
            ({', '.join(columns_with_types)});
            '''
        )

    def drop_table(self, table_name):
        self._execute(f'DROP TABLE {table_name};')

    def add(self, table_name, data):
        placeholders = ', '.join('?' * len(data))
        column_names = ', '.join(data.keys())
        column_values = tuple(data.values())

        self._execute(
            f'''
            INSERT INTO {table_name}
            ({column_names})
            VALUES ({placeholders});
            ''',
            column_values,
        )

    def delete(self, table_name, criteria):
        placeholders = [f'{column} = ?' for column in criteria.keys()]
        delete_criteria = ' AND '.join(placeholders)
        self._execute(
            f'''
            DELETE FROM {table_name}
            WHERE {delete_criteria};
            ''',
            tuple(criteria.values()),
        )

    def select(self, table_name, criteria=None, order_by=None):
        criteria = criteria or {}

        query = f'SELECT * FROM {table_name}'

        if criteria:
            placeholders = [f'{column} = ?' for column in criteria.keys()]
            select_criteria = ' AND '.join(placeholders)
            query += f' WHERE {select_criteria}'

        if order_by:
            query += f' ORDER BY {order_by}'

        return self._execute(
            query,
            tuple(criteria.values()),
        )

In [3]:
class BookmarksTable:

  @classmethod
  def create_bookmarks(cls):
    db.create_table('bookmarks', {
    'id': 'integer primary key autoincrement',
    'title': 'text not null',
    'url': 'text not null',
    'notes': 'text',
    'date_added': 'text not null',
  })

  @classmethod
  def add_bookmark(cls,data):
    data['date_added'] = datetime.utcnow().isoformat()
    db.add('bookmarks', data)
    return 'Bookmark added!'
  
  @classmethod
  def list_bookmarks(cls,order_by='date_added'):
    return db.select('bookmarks', order_by=order_by).fetchall()
  
  @classmethod
  def delete_bookmark(cls,data):
    db.delete('bookmarks', {'id': data})
    return 'Bookmark deleted!'

In [4]:
db = DatabaseManager('bookmarks.db')

In [5]:
bookmarks = BookmarksTable()

In [7]:
bookmarks.create_bookmarks()

In [8]:
bookmarks.add_bookmark({'title':'title_a',
                             'url':'url_a',
                             'notes':'note_a'})
bookmarks.add_bookmark({'title':'title_b',
                             'url':'url_b',
                             'notes':'note_b'})
bookmarks.add_bookmark({'title':'title_c',
                             'url':'url_c',
                             'notes':'note_c'})

'Bookmark added!'

In [9]:
bookmarks.list_bookmarks()

[(1, 'title_a', 'url_a', 'note_a', '2022-07-02T07:20:15.567232'),
 (2, 'title_b', 'url_b', 'note_b', '2022-07-02T07:20:15.577099'),
 (3, 'title_c', 'url_c', 'note_c', '2022-07-02T07:20:15.586380')]

In [10]:
bookmarks.delete_bookmark(3)

'Bookmark deleted!'

In [11]:
def select_sqlite(sql, path_database:str="/content/bookmarks.db")->pd.DataFrame:
    conn = sqlite3.connect(path_database)
    return pd.read_sql(sql, conn)

In [12]:
sql = """select * from bookmarks as b"""

In [13]:
select_sqlite(sql)

,id,title,url,notes,date_added
0,1,title_a,url_a,note_a,2022-07-02T07:20:15.567232
1,2,title_b,url_b,note_b,2022-07-02T07:20:15.577099
